
Code is an adapted version from [simple-kafka-python](https://github.com/quixio/simple-kafka-python/tree/main).

In [ ]:
pip install quixstreams

In [ ]:
from quixstreams import Application
import json
import time
# Application top level object

app = Application (
    broker_address = "localhost:19092",
    loglevel = "DEBUG",
    consumer_group = "stu_consumer_group",
    auto_offset_reset = "earliest"
)


max_records = 61  # Maximum number of records to process
start_time = time.time()  # Start time for elapsed time measurement
max_duration = 10 # max running time

records_processed = 0

with app.get_consumer() as consumer:
    consumer.subscribe(["stu-topic"]) # subscribing to a topic

    while True:
        msg = consumer.poll(1)
        # breakpoint() # add for debugging
        if msg is None:
            print("Waiting for message...")
        elif msg.error() is not None:
            raise Exception(msg.error())
        # else:
        #     breakpoint() # add for debugging
        else:
            key = msg.key().decode('utf8')
            value = json.loads(msg.value())
            offset = msg.offset()

            print(f"{offset} {'|'} {key} {'|'} {value} {'|'}")
            records_processed += 1

        if records_processed >= max_records or time.time() - start_time >= max_duration:
                print("Maximum records or time limit reached. Exiting...")
                break



Story of building this out from the [Quix video](https://youtu.be/eCsSAzTy5cE?si=kgE3gDAD-QqTVWZD).

# Start and inspecting `msg` properties
Started with:
```python
while True:
        msg = consumer.poll(1)
        breakpoint()
```
Checked what the value of msg was in the DEBUG CONSOLE (`print (msg)`), found it was `None` so built in handling for that with an IF.

`Continue`'d in debug mode, find the next value which was `<cimpl.Message object at 0x10605b8c0>` , not been stringified well. Use `dir(msg)` to get output as below, have trimmed out the middle. `error` is what is of interest. So call that with msg.error() and you get the details.

```python
00:
'__class__'
...
25:
'error'
```
I wanted to copy it out so used;

```python
a = msg.error()
with open("output.txt", "w") as file:
    # Write the variable to the file
    file.write(f"a = {a}\n")
```
But didn't need to do this, just put in generic error handling instead.

# Consume the messages
Add in block for consuming the key, the value and the offset.
Decode utf8 was for the string, json.loads is for load string to convert the json value into string.

# Timeout logic
I wanted it to exit after certain amount of message or time, so added in some logic there to count and exit (break).

# Issues faced
Trying to restart the code to do it again after adjusting things it was always waiting, I think this was to do with offsets not being reset. 
I tried in Console, deleting the consumer group, resetting offsets in the consumer group and emptying the offsets topics (this isn't allowed). None of this worked.
Restart cluster kinda worked but couldn't get the error stuff I tried on first pass. Adding in the `auto_offset_reset` worked.




# Custom Consumer
My consumer for the weather.

In [ ]:
from quixstreams import Application
import json
import time
# Application top level object

app = Application (
    broker_address = "localhost:19092",
    loglevel = "DEBUG",
    consumer_group = "stu_consumer_group_weather",
    auto_offset_reset = "earliest" # comment me in and out
)


max_records = 1000  # Maximum number of records to process
start_time = time.time()  # Start time for elapsed time measurement
max_duration = 100 # max running time

records_processed = 0

with app.get_consumer() as consumer:
    consumer.subscribe(["weather_data_demo"]) # subscribing to a topic

    while True:
        msg = consumer.poll(1)
        # breakpoint() # add for debugging
        if msg is None:
            print("Waiting for message...")
        elif msg.error() is not None:
            raise Exception(msg.error())
        # else:
        #     breakpoint() # add for debugging
        else:
            key = msg.key().decode('utf8')
            value = json.loads(msg.value())
            offset = msg.offset()

            print(f"{offset} {'|'} {key} {'|'} {value} {'|'}")
            records_processed += 1

        if records_processed >= max_records or time.time() - start_time >= max_duration:
                print("Maximum records or time limit reached. Exiting...")
                break
